In [14]:
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate
from scipy.special import erfc 
from scipy.optimize import root



filedat="sf_CDM__k500-z127-matterpower-original.dat"


dzeta = 12
Tvir = 10**4

xi=3

fileqv = "QV_CDM_zeta%s_xi%s_T%.2e_k_sharp.txt"%(dzeta,xi,Tvir)
filetau="tau_CDM_zeta%s5_xi%s_T%.2e_k_sharp.txt"%(dzeta,xi,Tvir)

klow=500 #h/Mpc
#(defining cosmology constants)
step=0.1

h = 0.678
Omegab = 0.0463
Omegam0 = 0.307
ns = 0.961
Tcmb = 2.725
sigm8 = 0.823
sigm8scal =0.9955

ro = 2.775*10**11 #(in h^2 Msun/Mpc^3)
rodm = Omegam0*ro
aa = 2.7 #parameter a in Lovell:15
mpc = 3.0856*10**24
csl = 2.99*10**10
SigmT = 6.6525*10**(-25)
Yp = 0.24667
Xp = 1 - Yp
mu = 1/(Xp + Yp/4)
muhalo=0.6
mp = 1.6726*10**(-27)
nH = nH = 1.98*10**30*Omegab*ro*(h**2)/(mpc**3*mp*mu)

sigm8 = 0.823
sigm8scal = 0.9955
SigmT = 6.6525*10**(-25)

OmegaM = Omegam0
OmegaLambda = 1.0 - Omegam0



g= lambda z: np.sqrt(OmegaM*(1+z)**3+OmegaLambda+(1-OmegaM-OmegaLambda)*(1+z)**2)
 
    
omegam=lambda z: OmegaM*((1 + z)**3)*(g(z))**(-2);
omegal= lambda z:OmegaLambda*(g(z))**(-2);
#Eisenstein & Hu'1998 approximation of the growth function: 
D1EH=lambda z: (1 + z)**(-1)*(5/2)*(omegam(z))/((omegam(z))**(4/7)-omegal(z) + (1 + 0.5*omegam(z))*(1 + omegal(z)/70));
DEH =lambda z: D1EH(z)/D1EH(0);




power_spectrum_dat=np.transpose(np.loadtxt(filedat))

#power_spectrum=interp1d(power_spectrum_dat[0],power_spectrum_dat[1], kind='linear')
R=lambda M: ((3*M/(4*np.pi*rodm))**(1/3))/aa

kc=lambda M: 1/R(M)

def sigma(M):
    kmax=kc(M)
    j=0
    for i,k in enumerate(power_spectrum_dat[0]):
        if k<=kmax:
            j=i
    add1=power_spectrum_dat[1,0]*power_spectrum_dat[0,0]**3/2
    add2=(power_spectrum_dat[1,j]*power_spectrum_dat[0,j]**2+power_spectrum_dat[1,j+1]*power_spectrum_dat[0,j+1]**2)*(kmax-power_spectrum_dat[0,j])**2/(power_spectrum_dat[0,j+1]-power_spectrum_dat[0,j])/2
    
    y=power_spectrum_dat[1,:j+1]*(power_spectrum_dat[0,:j+1])**2
    x=power_spectrum_dat[0,:j+1]
    return 1/(2*np.pi**2)*(sigm8/sigm8scal)**2*(integrate.simps(y,x)+add1+add2)

#sigma2=lambda M:1/(2*np.pi**2)*(sigm8/sigm8scal)**2*(integrate.quad(lambda x:power_spectrum(x)*x**2,power_spectrum_dat[0,0], kc(M)))[0]


deltaz=lambda z: 1.676/DEH(z)



Mmin=lambda z: 3.59*10**7*(((1+z)/10)**(-3/2))*(muhalo/0.6)**(-3/2)*(Tvir/(10**4))**(3/2)
smin=lambda z: sigma(Mmin(z))
Mj=lambda z: (5.7*10**3)*((Omegam0/0.15)**(-1/2))*(((Omegab*h**2)/0.02)**(-3/5))*((1 + z)/10)**(3/2)

Mlow=(4.*np.pi/3)*rodm*(aa/klow)**3;
def mj(z):
    if Mlow<Mj(z):
        return Mj(z)
    else: return Mlow

sj=lambda z: sigma(mj(z))

eq=lambda x,smin1,sj1,S: (dzeta+xi)*erfc((deltaz(z)-x)/np.sqrt(2*(smin1-S)))-xi*erfc((deltaz(z)-x)/np.sqrt(2*(sj1-S)))-1

import time
t1=time.clock()
nb=lambda sigm,B0,B1:np.sqrt(2/np.pi)*B0/sigm**2*np.exp(-(B0+B1*sigm**2)**2/(2*sigm**2))
z=20
Qv=0
Qv_arr=np.array([[None,None]])
while abs(Qv)<0.99:
    sj1=sj(z)
    smin1=smin(z)
    B0=root(eq,deltaz(z),args=(smin1,sj1,0))['x'][0]
    B2=root(eq,deltaz(z),args=(smin1,sj1,0.1))['x'][0]
    
    B1=(B2-B0)/0.1
    Qv=integrate.quad(nb,0,40,args=(B0,B1))[0]
    Qv_arr=np.append(Qv_arr,np.array([[z,abs(Qv)]]),axis=0)
    z=z-step
zr=Qv_arr[-1,0]


for i in range(0,round(zr/step)):
    if z<0:
        z=0
    Qv_arr=np.append(Qv_arr, np.array([[abs(z),1]]),axis=0)
    
    z=z-step



Qv_arr=Qv_arr[1:]
with open(fileqv,'wb') as fileout_qv:
    np.savetxt(fileout_qv,Qv_arr,delimiter=" ", fmt="%s")

Qv_arr=np.transpose(Qv_arr)


fe= np.vectorize(lambda x:1 + Yp/(2*Xp) if x<=4 else 1 + Yp/(4*Xp))


hz= lambda y:h*(10**7/mpc)*(Omegam0*(1 + y)**3 + (1 - Omegam0))**(0.5)

y=Qv_arr[1]*fe(Qv_arr[0])/hz(Qv_arr[0])*(1+Qv_arr[0])**2
tau=-nH*csl*SigmT*integrate.cumtrapz(y,Qv_arr[0])
t2=time.clock()
with open(filetau,'wb') as fileout_tau:
    np.savetxt(fileout_tau,np.vstack((Qv_arr[0,:0:-1],tau)).T,delimiter=" ", fmt="%s")
print(zr,tau[-1])
print('time %s' %(t2-t1))

6.699999999999978 0.08277149926376522
time 1.1845990803356017


C:\Users\1\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:364: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)
